In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import _pickle as cPickle
import cPickle
import sys
sys.path.insert(0, '..')
import tensorflow as tf

#from helper import plot_mats

X = np.load('./shiftsuniform_x.npy').astype('float32')
Y = np.load('./shiftsuniform_y.npy').astype('float32')

numvisX = X.shape[1]
numvisY = Y.shape[1]
ntrain = X.shape[0]

def random_binomial(shape, p=0.0, dtype=None, seed=None):
    """Returns a tensor with random binomial distribution of values.
    # Arguments
        shape: A tuple of integers, the shape of tensor to create.
        p: A float, `0. <= p <= 1`, probability of binomial distribution.
        dtype: String, dtype of returned tensor.
        seed: Integer, random seed.
    # Returns
        A tensor.
    """
    if dtype is None:
        dtype = 'float32'
    if seed is None:
        seed = np.random.randint(10e6)
    return tf.where(tf.random_uniform(shape, dtype=dtype, seed=seed) <= p,
                    tf.ones(shape, dtype=dtype),
                    tf.zeros(shape, dtype=dtype))

def dispims(M, height, width, border=0, bordercolor=0.0, layout=None, **kwargs):
    from pylab import cm, ceil
    numimages = M.shape[1]
    if layout is None:
        n0 = int(np.ceil(np.sqrt(numimages)))
        n1 = int(np.ceil(np.sqrt(numimages)))
    else:
        n0, n1 = layout
    im = bordercolor * np.ones(((height+border)*n0+border,(width+border)*n1+border),dtype='<f8')
    for i in range(n0):
        for j in range(n1):
            if i*n1+j < M.shape[1]:
                im[i*(height+border)+border:(i+1)*(height+border)+border,
                   j*(width+border)+border :(j+1)*(width+border)+border] = np.vstack((
                            np.hstack((np.reshape(M[:,i*n1+j],(height, width)),
                                   bordercolor*np.ones((height,border),dtype=float))),
                            bordercolor*np.ones((border,width+border),dtype=float)
                            ))
    pylab.imshow(im, cmap=cm.gray, interpolation='nearest', **kwargs)
    pylab.show()

    
    

numpy_rng = np.random.RandomState(1)

numfac  = 100
nummap  = 25

#NORMALIZE DATA:
X -= X.mean(0)[None, :]
Y -= Y.mean(0)[None, :]
X /= X.std(0)[None, :] + X.std() * 0.1
Y /= Y.std(0)[None, :] + Y.std() * 0.1


inputsX = tf.placeholder(tf.float32, [None, numvisX])
inputsY = tf.placeholder(tf.float32, [None, numvisY])

wxf = tf.Variable(tf.random_normal(shape=(numvisX, numfac)) * 0.01)
wyf = tf.Variable(tf.random_normal(shape=(numvisY, numfac)) * 0.01)

whf = tf.Variable(np.exp(numpy_rng.uniform(low=-3.0, high=-2.0, size=(nummap, numfac)),dtype='float32'))
whf_in = tf.Variable(numpy_rng.uniform(low=-0.01, high=+0.01, size=(nummap, numfac)).astype('float32'))

bmap = tf.Variable(np.zeros(nummap, dtype='float32'), name='bmap')
bvisX = tf.Variable(np.zeros(numvisX, dtype='float32'), name='bvisX')
bvisY = tf.Variable(np.zeros(numvisY, dtype='float32'), name='bvisY')


CorruptedX = tf.multiply(random_binomial(tf.shape(inputsX),p=0.5),inputsX)
CorruptedY = tf.multiply(random_binomial(tf.shape(inputsY),p=0.5),inputsY)

fx = tf.matmul(CorruptedX , wxf)
fy = tf.matmul(CorruptedY , wyf)

mappings = tf.sigmoid(tf.matmul(tf.multiply(fx , fy), tf.transpose(whf_in))+ bmap)
fH = tf.matmul(mappings, whf)

outputx = tf.matmul(tf.multiply(fy , fH),tf.transpose(wxf)) + bvisX
outputy = tf.matmul(tf.multiply(fx , fH),tf.transpose(wyf)) + bvisY
#output = tf.matmul(fH,tf.transpose(wxf))

cost = tf.nn.l2_loss(outputx-inputsX) + tf.nn.l2_loss(outputy-inputsY)

#+tf.reduce_sum(wxf**2)

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)




        

In [2]:
def normalizefilters(sess,_wxf,_wyf,center=True):
        def inplacemult(x, v):
            x[:, :] *= v
            return x
        def inplacesubtract(x, v):
            x[:, :] -= v
            return x
        wxf_ = _wxf.eval(sess)
        wyf_ = _wyf.eval(sess)
        nwxf = (wxf_.std(0)+SMALL)[np.newaxis, :]
        nwyf = (wyf_.std(0)+SMALL)[np.newaxis, :]
        meannxf = nwxf.mean()
        meannyf = nwyf.mean()

        # CENTER FILTERS 
        if center:
            _wxf.assign(inplacesubtract(wxf_, wxf_.mean(0)[np.newaxis,:])).eval()
            _wyf.assign(inplacesubtract(wyf_, wyf_.mean(0)[np.newaxis,:])).eval()
        # FIX STANDARD DEVIATION 
        _wxf.assign(inplacemult(wxf_, meannxf/nwxf)).eval()
        _wyf.assign(inplacemult(wyf_, meannyf/nwyf)).eval()

In [ ]:
wxf_normalized = tf.nn.l2_normalize(wxf, [0,1], epsilon=1e-12, name=None)
wyf_normalized = tf.nn.l2_normalize(wyf, [0,1], epsilon=1e-12, name=None)

normalize_wxf = wxf.assign(wxf_normalized)
normalize_wyf = wxf.assign(wyf_normalized)

In [ ]:


training_epochs = 3000
batch_size = 100
SMALL = 0.000001


with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for epoch in range(training_epochs):
        randidx  = np.random.randint(ntrain, size=batch_size).astype('int32')
        batch_xs = X[randidx]
        batch_ys = Y[randidx]  
        sess.run(optimizer, feed_dict={inputsX: batch_xs, inputsY: batch_ys})
        sess.run(normalize_wxf)
        sess.run(normalize_wyf)
#         normalizefilters(sess,wxf,wyf)
        
        cost_ = sess.run(cost, feed_dict={inputsX: X, inputsY: Y}) / ntrain
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch,training_epochs ,cost_) ) 

    import pylab
    pylab.subplot(1, 2, 1)
    dispims(wxf.eval(sess), 13, 13, 2)
    pylab.subplot(1, 2, 2)
    dispims(wyf.eval(sess), 13, 13, 2)

Epoch: 000/3000 cost: 139.652900000
Epoch: 001/3000 cost: 139.652650000
Epoch: 002/3000 cost: 139.655887500
Epoch: 003/3000 cost: 139.659012500
Epoch: 004/3000 cost: 139.648475000
Epoch: 005/3000 cost: 139.641975000
Epoch: 006/3000 cost: 139.628825000
Epoch: 007/3000 cost: 139.613700000
Epoch: 008/3000 cost: 139.599237500
Epoch: 009/3000 cost: 139.577050000
Epoch: 010/3000 cost: 139.557575000
Epoch: 011/3000 cost: 139.526625000
Epoch: 012/3000 cost: 139.509475000
Epoch: 013/3000 cost: 139.477850000
Epoch: 014/3000 cost: 139.453325000
Epoch: 015/3000 cost: 139.425687500
Epoch: 016/3000 cost: 139.397100000
Epoch: 017/3000 cost: 139.372275000
Epoch: 018/3000 cost: 139.349387500
Epoch: 019/3000 cost: 139.319875000
Epoch: 020/3000 cost: 139.290762500
Epoch: 021/3000 cost: 139.249712500
Epoch: 022/3000 cost: 139.234462500
Epoch: 023/3000 cost: 139.210875000
Epoch: 024/3000 cost: 139.179000000
Epoch: 025/3000 cost: 139.165300000
Epoch: 026/3000 cost: 139.148337500
Epoch: 027/3000 cost: 139.12

Epoch: 229/3000 cost: 136.613025000
Epoch: 230/3000 cost: 136.591900000
Epoch: 231/3000 cost: 136.541675000
Epoch: 232/3000 cost: 136.491725000
Epoch: 233/3000 cost: 136.473637500
Epoch: 234/3000 cost: 136.414975000
Epoch: 235/3000 cost: 136.412112500
Epoch: 236/3000 cost: 136.350175000
Epoch: 237/3000 cost: 136.326725000
Epoch: 238/3000 cost: 136.290350000
Epoch: 239/3000 cost: 136.268937500
Epoch: 240/3000 cost: 136.230550000
Epoch: 241/3000 cost: 136.196012500
Epoch: 242/3000 cost: 136.159650000
Epoch: 243/3000 cost: 136.103975000
Epoch: 244/3000 cost: 136.092075000
Epoch: 245/3000 cost: 136.040650000
Epoch: 246/3000 cost: 136.017737500
Epoch: 247/3000 cost: 135.975712500
Epoch: 248/3000 cost: 135.945775000
Epoch: 249/3000 cost: 135.928150000
Epoch: 250/3000 cost: 135.915137500
Epoch: 251/3000 cost: 135.889750000
Epoch: 252/3000 cost: 135.858400000
Epoch: 253/3000 cost: 135.808187500
Epoch: 254/3000 cost: 135.760875000
Epoch: 255/3000 cost: 135.754100000
Epoch: 256/3000 cost: 135.70

Epoch: 459/3000 cost: 131.648225000
Epoch: 460/3000 cost: 131.645325000
Epoch: 461/3000 cost: 131.648950000
Epoch: 462/3000 cost: 131.629825000
Epoch: 463/3000 cost: 131.639412500
Epoch: 464/3000 cost: 131.619350000
Epoch: 465/3000 cost: 131.567812500
Epoch: 466/3000 cost: 131.577562500
Epoch: 467/3000 cost: 131.572650000
Epoch: 468/3000 cost: 131.534800000
Epoch: 469/3000 cost: 131.551450000
Epoch: 470/3000 cost: 131.512800000
Epoch: 471/3000 cost: 131.510537500
Epoch: 472/3000 cost: 131.489575000
Epoch: 473/3000 cost: 131.512075000
Epoch: 474/3000 cost: 131.494700000
Epoch: 475/3000 cost: 131.473925000
Epoch: 476/3000 cost: 131.512850000
Epoch: 477/3000 cost: 131.453175000
Epoch: 478/3000 cost: 131.472725000
Epoch: 479/3000 cost: 131.449012500
Epoch: 480/3000 cost: 131.453675000
Epoch: 481/3000 cost: 131.447000000
Epoch: 482/3000 cost: 131.447575000
Epoch: 483/3000 cost: 131.448575000
Epoch: 484/3000 cost: 131.480750000
Epoch: 485/3000 cost: 131.418600000
Epoch: 486/3000 cost: 131.40